In [2]:
import os
os.chdir("..")
!pwd
!pyenv version
%load_ext autoreload
%autoreload 2

/Users/loicmorel/code/projetcs/crypto_assistant
crypto_assistant (set by PYENV_VERSION environment variable)


In [23]:
import pandas as pd
import numpy as np
from app.data_collection.datamgt import ohlcv_from_csv_to_df
from tqdm import trange
from scipy import stats
from sklearn.linear_model import LinearRegression

In [5]:
file = 'data/ohlcv_VITEUSDT_1h_from_1576245600000_to_1651507200000.csv'
data = ohlcv_from_csv_to_df(file)
data

,o_ts,Open,High,Low,Close,Volume,c_ts,o_date
0,2019-12-13 14:00:00,0.02083,0.02400,0.01522,0.01643,29204100.1,2019-12-13 14:59:59.999000064,18243.583333
1,2019-12-13 15:00:00,0.01643,0.01674,0.01556,0.01601,11152260.3,2019-12-13 15:59:59.999000064,18243.625000
2,2019-12-13 16:00:00,0.01602,0.01648,0.01576,0.01589,4090513.3,2019-12-13 16:59:59.999000064,18243.666667
3,2019-12-13 17:00:00,0.01589,0.01648,0.01586,0.01629,1789868.9,2019-12-13 17:59:59.999000064,18243.708333
4,2019-12-13 18:00:00,0.01629,0.01692,0.01609,0.01625,5526217.3,2019-12-13 18:59:59.999000064,18243.750000
...,...,...,...,...,...,...,...,...
20872,2022-05-02 12:00:00,0.04070,0.04096,0.04070,0.04084,185716.7,2022-05-02 12:59:59.999000064,19114.500000
20873,2022-05-02 13:00:00,0.04082,0.04115,0.04018,0.04107,1659280.1,2022-05-02 13:59:59.999000064,19114.541667
20874,2022-05-02 14:00:00,0.04105,0.04133,0.04073,0.04108,355018.9,2022-05-02 14:59:59.999000064,19114.583333
20875,2022-05-02 15:00:00,0.04116,0.04116,0.04071,0.04075,642934.6,2022-05-02 15:59:59.999000064,19114.625000


In [24]:
data_ = process_data(data, slope=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20075/20075 [00:09<00:00, 2144.68it/s]


In [25]:
data_

,o_ts,Open,High,Low,Close,Volume,c_ts,o_date,diff,direction,ushadow_width,lshadow_width,body_per,upper_per,lower_per,slope
6,2019-12-13 20:00:00,0.01579,0.01631,0.01559,0.01608,1303358.1,2019-12-13 20:59:59.999000064,18243.833333,0.00029,1.0,0.00023,0.00020,55.788294,99.980075,99.990037,-0.000041
7,2019-12-13 21:00:00,0.01607,0.01629,0.01586,0.01592,827305.8,2019-12-13 21:59:59.999000064,18243.875000,-0.00015,-1.0,0.00022,0.00006,39.384807,99.975093,99.975093,-0.000014
8,2019-12-13 22:00:00,0.01592,0.01617,0.01572,0.01573,820948.8,2019-12-13 22:59:59.999000064,18243.916667,-0.00019,-1.0,0.00025,0.00001,44.859278,99.980075,99.965131,-0.000050
9,2019-12-13 23:00:00,0.01598,0.01680,0.01595,0.01658,3373578.7,2019-12-13 23:59:59.999000064,18243.958333,0.00060,1.0,0.00022,0.00003,71.870486,99.967621,99.967621,-0.000001
10,2019-12-14 00:00:00,0.01658,0.01670,0.01619,0.01627,1226442.6,2019-12-14 00:59:59.999000064,18244.000000,-0.00031,-1.0,0.00012,0.00008,57.163138,99.955168,99.975093,0.000047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20070,2022-05-02 12:00:00,0.04070,0.04096,0.04070,0.04084,185716.7,2022-05-02 12:59:59.999000064,19114.500000,0.00014,1.0,0.00012,0.00000,27.450980,56.862745,5.882353,-0.000130
20071,2022-05-02 13:00:00,0.04082,0.04115,0.04018,0.04107,1659280.1,2022-05-02 13:59:59.999000064,19114.541667,0.00025,1.0,0.00008,0.00064,48.039216,41.176471,96.078431,-0.000071
20072,2022-05-02 14:00:00,0.04105,0.04133,0.04073,0.04108,355018.9,2022-05-02 14:59:59.999000064,19114.583333,0.00003,1.0,0.00025,0.00032,3.921569,82.352941,92.156863,-0.000019
20073,2022-05-02 15:00:00,0.04116,0.04116,0.04071,0.04075,642934.6,2022-05-02 15:59:59.999000064,19114.625000,-0.00041,-1.0,0.00000,0.00004,60.784314,6.862745,27.450980,-0.000003


In [20]:
def process_data(data, slope=True):
    '''Including calculation of CLUR, Quartiles, and cus trend
    Args:
        data (dataframe): csv data from assets. With column names open, high, low, close.

    Returns:
        dataframe.
    '''
    if slope:
        # process slpoe
        data['diff'] = data['Close'] - data['Open']
        data = data.query('diff != 0').reset_index(drop=True)
        data['direction'] = np.sign(data['diff'])
        data['ushadow_width'] = 0
        data['lshadow_width'] = 0

        for idx in trange(len(data)):
            if data.loc[idx, 'direction'] == 1:
                data.loc[idx, 'ushadow_width'] = data.loc[idx, 'High'] - data.loc[idx, 'Close']
                data.loc[idx, 'lshadow_width'] = data.loc[idx, 'Open'] - data.loc[idx, 'Low']
            else:
                data.loc[idx, 'ushadow_width'] = data.loc[idx, 'High'] - data.loc[idx, 'Open']
                data.loc[idx, 'lshadow_width'] = data.loc[idx, 'Close'] - data.loc[idx, 'Low']

            if idx <= 50:
                data.loc[idx, 'body_per'] = stats.percentileofscore(abs(data['diff']), abs(data.loc[idx,'diff']), 'rank')
                data.loc[idx, 'upper_per'] = stats.percentileofscore(data['ushadow_width'], data.loc[idx,'ushadow_width'], 'rank')
                data.loc[idx, 'lower_per'] = stats.percentileofscore(data['lshadow_width'], data.loc[idx,'lshadow_width'], 'rank')
            else:
                data.loc[idx, 'body_per'] = stats.percentileofscore(abs(data.loc[idx-50:idx, 'diff']),abs(data.loc[idx, 'diff']), 'rank')
                data.loc[idx, 'upper_per'] = stats.percentileofscore(data.loc[idx-50:idx, 'ushadow_width'], data.loc[idx, 'ushadow_width'], 'rank')
                data.loc[idx, 'lower_per'] = stats.percentileofscore(data.loc[idx-50:idx, 'lshadow_width'], data.loc[idx, 'lshadow_width'], 'rank')

        data['slope'] = data['Close'].rolling(7).apply(get_slope, raw=False)
        data.dropna(inplace=True)
    else:
        # process trend
        data['trend'] = data['slope'].rolling(1).apply(get_trend, raw=False)
        data['previous_trend'] = data['trend'].shift(1).fillna(0)
    return data

In [19]:
def get_slope(series):
    y = series.values.reshape(-1, 1)
    x = np.array(range(1, series.shape[0] + 1)).reshape(-1,1)
    model = LinearRegression()
    model.fit(x, y)
    slope = model.coef_
    return slope


def get_trend(slope):
    '''Need to run `process_data` first with slope only, then calculate by yourself.
    25 percentile: 7.214285714286977e-05
    '''
    slope = np.array(slope)
    thres = 7.214285714286977e-05
    if (slope >= thres):
        return 1
    elif (slope <= -thres):
        return -1
    else:
        return 0